## Курсовая Работа

### 1. Алгоритм в лоб

In [ ]:
from datetime import datetime, timedelta
import random
import pandas as pd
from tabulate import tabulate

NUM_BUSES = 4
HALF_NUM_BUSES = 2
WEEKDAYS = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница']
WEEKENDS = ['Суббота', 'Воскресенье']
SHIFT_8_DURATION = timedelta(hours=8)
SHIFT_12_DURATION = timedelta(hours=12)
BREAK_LONG_END = timedelta(minutes=45)
BREAK_MIN = timedelta(minutes=15)
BREAK_MIN_REST = timedelta(minutes=105)
BREAK_INTERVAL = timedelta(minutes=120)
BREAK_HOUR = timedelta(hours=1)

class Driver:
    def __init__(self, driver_id, shift_type):
        self.driver_id = driver_id
        self.shift_type = shift_type
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

class Bus:
    def __init__(self, bus_id, bus_index):
        self.bus_id = bus_id
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.drivers = []
        self.bus_index = bus_index

drivers_count = 0
drivers = []
buses = [Bus(f"Автобус {i+1}", i) for i in range(NUM_BUSES)]

for driver in drivers:
    driver.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
    driver.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

for bus in buses:
    bus.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}

def time_intervals(start_time, shift_duration):
    start = datetime.strptime(start_time, "%H:%M")
    end = start + shift_duration
    return start, end

def set_breaks(start, end, break_type):
    breaks = []
    
    if break_type == "15_мин":
        current = start + BREAK_MIN_REST
        break_duration = BREAK_MIN
        while current + break_duration < end:
            breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M"), "15_мин"))
            current += BREAK_INTERVAL

    elif break_type == "1_час":
        if start.strftime("%H:%M") == "09:00":
            long_start = datetime.strptime("14:15", "%H:%M")
            long_end = long_start + BREAK_LONG_END
        else:
            long_start = datetime.strptime("13:00", "%H:%M")
            long_end = long_start + BREAK_HOUR
        if start <= long_start < end:
            breaks.append((long_start.strftime("%H:%M"), long_end.strftime("%H:%M"), "1_час"))
        elif long_start <= start < long_end:
            breaks.append((start.strftime("%H:%M"), (start + BREAK_HOUR).strftime("%H:%M"), "1_час"))
    return breaks

def set_8_drivers():
    global drivers_count

    for bus_index, bus in enumerate(buses):
        if bus_index < HALF_NUM_BUSES:
            shift_start_time = "06:00"
        else:
            shift_start_time = "09:00"

        drivers_count += 1
        driver = Driver(f"Водитель {drivers_count} (8 час)", "8_час")
        drivers.append(driver)

        for day in WEEKDAYS:
            various_start_time = (datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % HALF_NUM_BUSES) * 15)).strftime("%H:%M")
            start, end = time_intervals(various_start_time, SHIFT_8_DURATION)

            if "09:00" in various_start_time:
                breaks = set_breaks(start, end, "1_час")
            else:
                breaks = set_breaks(start, end, "15_мин")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

def set_12_drivers():
    global drivers_count
    rest_12_counter = {}

    for _, bus in enumerate(buses): 
        for _ in range(3):
            drivers_count += 1
            driver = Driver(f"Водитель {drivers_count} (12 час)", "12_час")
            bus.drivers.append(driver)
            drivers.append(driver)
            rest_12_counter[driver.driver_id] = 0

    for bus in buses:
        driver_index = 0

        for day in WEEKDAYS + WEEKENDS:
            while rest_12_counter[bus.drivers[driver_index].driver_id] > 0:
                rest_12_counter[bus.drivers[driver_index].driver_id] -= 1
                driver_index = (driver_index + 1) % len(bus.drivers)

            driver = bus.drivers[driver_index]

            if day in WEEKDAYS:
                if bus.bus_index < HALF_NUM_BUSES:
                    various_start_time = (datetime.strptime("14:00", "%H:%M") + timedelta(minutes=bus.bus_index * 15)).strftime("%H:%M")
                else:
                    various_start_time = (datetime.strptime("17:00", "%H:%M") + timedelta(minutes=(bus.bus_index - HALF_NUM_BUSES) * 15)).strftime("%H:%M")
            else:
                various_start_time = (datetime.strptime("06:00", "%H:%M") + timedelta(minutes=bus.bus_index * 15)).strftime("%H:%M")

            start, end = time_intervals(various_start_time, SHIFT_12_DURATION)

            if day in WEEKDAYS + WEEKENDS:
                if start.hour == 6 and start.minute == 0:
                    breaks = set_breaks(start, end, "1_час")
                else:
                    breaks = set_breaks(start, end, "15_мин")
            else:
                breaks = set_breaks(start, end, "15_мин")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

            rest_12_counter[driver.driver_id] = 2

            driver_index = (driver_index + 1) % len(bus.drivers)

def set_weekend_12_drivers():
    global drivers_count

    for _, bus in enumerate(buses):
        for day in WEEKENDS:
            drivers_count += 1
            driver = Driver(f"Водитель {drivers_count} (12 час)", f"12_час Выходной {day}")
            bus.drivers.append(driver)
            drivers.append(driver)

            start_time = (datetime.strptime("18:00", "%H:%M") + timedelta(minutes=bus.bus_index * 15)).strftime("%H:%M")
            start, end = time_intervals(start_time, SHIFT_12_DURATION)

            breaks = set_breaks(start, end, "15_мин")
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

set_8_drivers()
set_12_drivers()
set_weekend_12_drivers()

def extract_breaks_data(drivers):
    long_breaks_data = {"Водитель": []}
    short_breaks_data = {"Водитель": []}

    for day in WEEKDAYS + WEEKENDS:
        long_breaks_data[day] = []
        short_breaks_data[day] = []

    for driver in drivers:
        long_breaks_day = []
        short_breaks_day = []

        for day in WEEKDAYS + WEEKENDS:
            long_breaks = [f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "1_час"]
            short_breaks = [f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "15_мин"]
            long_breaks_day.append("; ".join(long_breaks) if long_breaks else "")
            short_breaks_day.append("; ".join(short_breaks) if short_breaks else "")

        if any(long_breaks_day):
            long_breaks_data["Водитель"].append(driver.driver_id)
            for day, breaks in zip(WEEKDAYS + WEEKENDS, long_breaks_day):
                long_breaks_data[day].append(breaks)

        if any(short_breaks_day):
            short_breaks_data["Водитель"].append(driver.driver_id)
            for day, breaks in zip(WEEKDAYS + WEEKENDS, short_breaks_day):
                short_breaks_data[day].append(breaks)

    long_breaks_df = pd.DataFrame(long_breaks_data)
    short_breaks_df = pd.DataFrame(short_breaks_data)

    return long_breaks_df, short_breaks_df

long_breaks_df, short_breaks_df = extract_breaks_data(drivers)
general_schedule = []

for bus in buses:
    bus_row = [str(bus.bus_id)] + [""] * (len(WEEKDAYS + WEEKENDS))
    general_schedule.append(bus_row)
    uniq_drivers = set()

    for day in WEEKDAYS + WEEKENDS:
        for _, _, driver_id in bus.schedule.get(day, []):
            uniq_drivers.add(driver_id)

    for driver_id in sorted(uniq_drivers):
        driver_row = [str(driver_id)]

        for day in WEEKDAYS + WEEKENDS:
            shifts = [f"{start.strftime('%H:%M')} - {end.strftime('%H:%M')}" for start, end, driver in bus.schedule.get(day, []) if driver == driver_id]
            driver_row.append(", ".join(shifts) if shifts else "")
        
        general_schedule.append(driver_row)

columns = ["Водитель"] + WEEKDAYS + WEEKENDS
bus_schedule_df = pd.DataFrame(general_schedule, columns=columns)

long_breaks_filter = long_breaks_df[long_breaks_df.drop(columns=["Водитель"]).notna().any(axis=1)]
short_breaks_filter = short_breaks_df[short_breaks_df.drop(columns=["Водитель"]).notna().any(axis=1)]

long_rows = [["Долгие перерывы"] + [""] * (len(columns) - 1)] + long_breaks_filter.values.tolist()
short_break_rows = [["Короткие перерывы"] + [""] * (len(columns) - 1)] + short_breaks_filter.values.tolist()

general_breaks_rows = long_rows + short_break_rows
breaks_final_df = pd.DataFrame(general_breaks_rows, columns=columns)

def set_custom_index(df):
    custom_index = []
    counter = 1

    for row in df.itertuples(index=False):
        if row[0] and (row[0].startswith("Автобус ") or row[0] in ["Долгие перерывы", "Короткие перерывы"]):
            custom_index.append("")
        else:
            custom_index.append(counter)
            counter += 1

    df.index = custom_index

    return df

bus_schedule_df = set_custom_index(bus_schedule_df)
breaks_final_df = set_custom_index(breaks_final_df)

print(f"Всего водителей: {drivers_count}")

print("\nРасписание автобусов:")
print(tabulate(bus_schedule_df, headers='keys', tablefmt='grid'))

print("\nРасписание перерывов:")
print(tabulate(breaks_final_df, headers='keys', tablefmt='grid'))

Всего водителей: 24

Расписание автобусов:
+----+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|    | Водитель             | Понедельник   | Вторник       | Среда         | Четверг       | Пятница       | Суббота       | Воскресенье   |
+====+======================+===============+===============+===============+===============+===============+===============+===============+
|    | Автобус 1            |               |               |               |               |               |               |               |
+----+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
| 1  | Водитель 1 (8 час)   | 06:00 - 14:00 | 06:00 - 14:00 | 06:00 - 14:00 | 06:00 - 14:00 | 06:00 - 14:00 |               |               |
+----+----------------------+---------------+---------------+---------------+---------------+------------

### 2. Генетический Алгоритм

In [14]:
POP_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.01

drivers_count = 0
drivers = []

def random_schedule():
    global drivers_count
    rest_12_counter = {driver.driver_id: 0 for driver in drivers if driver.shift_type == "12_час"}
    schedule = []
    
    for bus_index, bus in enumerate(buses):
        for day in WEEKDAYS:
            shift_type = "8_час"
            shift_start_time = "06:00" if bus_index < HALF_NUM_BUSES else "09:00"

            various_start_time = (datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % HALF_NUM_BUSES) * 15)).strftime("%H:%M")

            suit_driver = [d for d in drivers if d.shift_type == "8_час"]
            driver = random.choice(suit_driver)

            shift_hours = SHIFT_8_DURATION
            start, end = time_intervals(various_start_time, shift_hours)

            breaks = set_breaks(start, end, "1_час" if "09:00" in various_start_time else "15_мин")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

            schedule.append((bus.bus_id, day, start, end, driver.driver_id))

    for bus_index, bus in enumerate(buses):
        for day in WEEKDAYS + WEEKENDS:
            if day in WEEKENDS:
                if bus_index < HALF_NUM_BUSES:
                    shift_type = "12_час (Выходной Суббота)" if day == "Суббота" else "12_час (Выходной Воскресенье)"
                    shift_start_time = "06:00"
                else:
                    shift_type = f"12_час (Выходной {day})"
                    shift_start_time = "18:00"
            elif day in WEEKDAYS:
                shift_type = "12_час"
                shift_start_time = "14:00" if bus_index < HALF_NUM_BUSES else "17:00"
            various_start_time = (datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % HALF_NUM_BUSES) * 15)).strftime("%H:%M")
            if shift_type == "12_час (Выходной Суббота)":
                suit_driver = [d for d in drivers if d.shift_type == "12_час (Выходной Суббота)"]
            elif shift_type == "12_час (Выходной Воскресенье)":
                suit_driver = [d for d in drivers if d.shift_type == "12_час (Выходной Воскресенье)"]
            elif shift_type == "12_час":
                suit_driver = [d for d in drivers if d.shift_type == "12_час" and rest_12_counter.get(d.driver_id, 0) == 0]
            if suit_driver:
                driver = random.choice(suit_driver)
            else:
                continue

            shift_hours = SHIFT_12_DURATION
            start, end = time_intervals(various_start_time, shift_hours)
            if day in WEEKENDS and start.hour == 6 and start.minute == 0:
                breaks = set_breaks(start, end, "1_час")
            else:
                breaks = set_breaks(start, end, "15_мин")
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))
            schedule.append((bus.bus_id, day, start, end, driver.driver_id))
    return schedule

def init_population(pop_size):
    population = []

    for _ in range(pop_size):
        individual = random_schedule()
        population.append(individual)
    return population

def fitness(schedule):
    penalty = 0
    driver_shifts = {driver.driver_id: {day: [] for day in WEEKDAYS + WEEKENDS} for driver in drivers}

    for _, day, start, end, driver_id in schedule:
        driver_shifts[driver_id][day].append((start, end))

    for driver_id, shifts in driver_shifts.items():
        for day, day_shifts in shifts.items():
            day_shifts.sort()

            for i in range(len(day_shifts)):
                for j in range(i + 1, len(day_shifts)):
                    if day_shifts[i][1] > day_shifts[j][0] and day_shifts[i][0] < day_shifts[j][1]:
                        penalty += 10

    for driver_id, shifts in driver_shifts.items():
        work_days = sum(1 for day, day_shifts in shifts.items() if day_shifts)

        if work_days > 5:
            penalty += 20

    return penalty

def selection(population, fitness_scores):
    selected = random.choices(population, weights=[max(fitness_scores) - score for score in fitness_scores], k=2)
    return selected

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return child1, child2

def mutation(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = random_schedule()[i]
    
    return individual

def genetic_algorithm(pop_size, generations, mutation_rate):
    population = init_population(pop_size)
    
    for _ in range(generations):
        fitness_scores = [fitness(individual) for individual in population]
        new_population = []

        for _ in range(pop_size // 2):
            parent1, parent2 = selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutation(child1, mutation_rate), mutation(child2, mutation_rate)])

        population = new_population

    best_individual = min(population, key=fitness)

    return best_individual

def set_drivers():
    global drivers_count

    for _, bus in enumerate(buses):
        for _ in range(3):
            drivers_count += 1
            driver = Driver(f"Водитель {drivers_count} (12 час)", "12_час")
            bus.drivers.append(driver)
            drivers.append(driver)

        for _ in range(2):
            drivers_count += 1
            driver = Driver(f"Водитель {drivers_count} (8 час)", "8_час")
            bus.drivers.append(driver)
            drivers.append(driver)

        
        drivers_count += 1
        driver = Driver(f"Водитель {drivers_count} (12 час)", "12_час (Выходной Суббота)")
        bus.drivers.append(driver)
        drivers.append(driver)

        drivers_count += 1
        driver = Driver(f"Водитель {drivers_count} (12 час)", "12_час (Выходной Воскресенье)")
        bus.drivers.append(driver)
        drivers.append(driver)

set_drivers()

best_schedule = genetic_algorithm(pop_size=POP_SIZE, generations=GENERATIONS, mutation_rate=MUTATION_RATE)

def format_table(schedule):
    bus_schedule = {bus.bus_id: {day: [] for day in WEEKDAYS + WEEKENDS} for bus in buses}
    driver_schedule = {driver.driver_id: {day: [] for day in WEEKDAYS + WEEKENDS} for driver in drivers}

    for bus_id, day, start, end, driver_id in schedule:
        bus_schedule[bus_id][day].append((start, end, driver_id))
        driver_schedule[driver_id][day].append((start, end, bus_id))

    return bus_schedule, driver_schedule

bus_schedule, driver_schedule = format_table(best_schedule)

def extract_breaks_data(drivers):
    long_breaks_data = {"Водитель": []}
    short_breaks_data = {"Водитель": []}

    for day in WEEKDAYS + WEEKENDS:
        long_breaks_data[day] = []
        short_breaks_data[day] = []

    for driver in drivers:
        long_breaks_day = []
        short_breaks_day = []

        for day in WEEKDAYS + WEEKENDS:
            if driver_schedule[driver.driver_id][day]:
                long_breaks = [f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "1_час"]
                short_breaks = [f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "15_мин"]
                long_breaks_day.append("; ".join(long_breaks) if long_breaks else "")
                short_breaks_day.append("; ".join(short_breaks) if short_breaks else "")
            else:
                long_breaks_day.append("")
                short_breaks_day.append("")

        if any(long_breaks_day):
            long_breaks_data["Водитель"].append(driver.driver_id)

            for day, breaks in zip(WEEKDAYS + WEEKENDS, long_breaks_day):
                long_breaks_data[day].append(breaks)

        if any(short_breaks_day):
            short_breaks_data["Водитель"].append(driver.driver_id)

            for day, breaks in zip(WEEKDAYS + WEEKENDS, short_breaks_day):
                short_breaks_data[day].append(breaks)

    long_breaks_df = pd.DataFrame(long_breaks_data)
    short_breaks_df = pd.DataFrame(short_breaks_data)

    return long_breaks_df, short_breaks_df

long_breaks_df, short_breaks_df = extract_breaks_data(drivers)
general_schedule = []

for bus in buses:
    bus_id = bus.bus_id
    bus_row = [str(bus_id)] + [""] * len(WEEKDAYS + WEEKENDS)
    general_schedule.append(bus_row)

    for driver_id, driver_shifts in driver_schedule.items():
        driver_row = [str(driver_id)]
        include_driver = False

        for day in WEEKDAYS + WEEKENDS:
            if day in driver_shifts:
                shifts = [f"{start.strftime('%H:%M')} - {end.strftime('%H:%M')}" for start, end, bus in driver_shifts[day] if bus == bus_id]
            
                if shifts:
                    include_driver = True
                    driver_row.append(", ".join(shifts))
                else:
                    driver_row.append("")
            else:
                driver_row.append("")
        
        if include_driver:
            general_schedule.append(driver_row)

columns = ["Водитель"] + WEEKDAYS + WEEKENDS
bus_schedule_df = pd.DataFrame(general_schedule, columns=columns)

long_breaks_filter = long_breaks_df[long_breaks_df.drop(columns=["Водитель"]).notna().any(axis=1)]
short_breaks_filter = short_breaks_df[short_breaks_df.drop(columns=["Водитель"]).notna().any(axis=1)]

long_rows = [["Долгие перерывы"] + [""] * (len(columns) - 1)] + long_breaks_filter.values.tolist()
short_break_rows = [["Короткие перерывы"] + [""] * (len(columns) - 1)] + short_breaks_filter.values.tolist()

general_breaks_rows = long_rows + short_break_rows
breaks_final_df = pd.DataFrame(general_breaks_rows, columns=columns)

def set_custom_index(df):
    custom_index = []
    counter = 1

    for row in df.itertuples(index=False):
        if row[0] and (row[0].startswith("Автобус ") or row[0] in ["Долгие перерывы", "Короткие перерывы"]):
            custom_index.append("")
        else:
            custom_index.append(counter)
            counter += 1

    df.index = custom_index

    return df

bus_schedule_df = set_custom_index(bus_schedule_df)
breaks_final_df = set_custom_index(breaks_final_df)

print(f"Всего водителей: {drivers_count}")

print("\nРасписание автобусов:")
print(tabulate(bus_schedule_df, headers='keys', tablefmt='grid'))

print("\nРасписание перерывов:")
print(tabulate(breaks_final_df, headers='keys', tablefmt='grid'))

Всего водителей: 28

Расписание автобусов:
+----+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|    | Водитель             | Понедельник   | Вторник       | Среда         | Четверг       | Пятница       | Суббота       | Воскресенье   |
+====+======================+===============+===============+===============+===============+===============+===============+===============+
|    | Автобус 1            |               |               |               |               |               |               |               |
+----+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
| 1  | Водитель 1 (12 час)  | 14:00 - 02:00 |               |               |               |               |               |               |
+----+----------------------+---------------+---------------+---------------+---------------+------------